<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**План по выполнению проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<br/>    
<div class="alert alert-info">
<h2> <a class="tocSkip"> </h2>

  Здесь у меня только одна обученная модель - CatBoost. На М1 не смог найти костыли, чтобы запрягать GPU для Bert и такие библотеки как Keras не устанавливались. Поэтому обучил отдельно на Colab.
    
Так же обучил логистическую регрессию с обработкой TFIDF
</div> 

<br/>    
<div class="alert alert-info">
<h2>  <a class="tocSkip"> </h2>

<b>👋:</b> Вот ссылки на логистическую регрессию с TFIDF https://colab.research.google.com/drive/10iAT5AiVl72VKIRkHiP6O1GpQ9TF0lDN?usp=sharing
 И на BERT на блокнот Насти Яниной из этого видео https://youtu.be/LmLDYP5yF-o) https://colab.research.google.com/drive/1QGuo_OAYkH8Re7DsP1r7rZfJ4De7BogD?usp=sharing
</div> 

## Подготовка

from google.colab import drive
drive.mount('/content/drive')

In [1]:
#!pip install catboost
import sys
!"{sys.executable}" -m pip install pytorch-transformers
!"{sys.executable}" -m pip install keras
!"{sys.executable}" -m pip install spacy

In [2]:
# импортируем библиотеки
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score


from catboost import CatBoostClassifier

from tqdm import notebook,tqdm, trange
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

 
nltk.download('wordnet','stopwords','punkt','averaged_perceptron_tagger')
from nltk.corpus import wordnet

import time 
 
import warnings
warnings.filterwarnings('ignore')


from pytorch_transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification

from IPython.display import clear_output
import matplotlib.pyplot as plt
import io

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from keras.preprocessing.sequence import pad_sequences

#from keras.utils import pad_sequences
#import spacy


In [3]:
# загрузим дадасет
try:
    df = pd.read_csv('/Users/gazizovilsur26/Downloads/toxic_comments.csv')
except:
    df = pd.read_csv('datasets/toxic_comments.csv')



In [4]:
# выведем на экран первые 10 строк
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [5]:
display(df.loc[1, :])

Unnamed: 0                                                    1
text          D'aww! He matches this background colour I'm s...
toxic                                                         0
Name: 1, dtype: object

In [6]:
# удалим столбец Unnamed: 0
df = df.drop('Unnamed: 0', axis=1)

In [7]:
# выведем общую информацию про датасет
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


**Промежуточный вывод:**
1. Пропусков нет
2. С типами данных всё хорошо
3. Столбец Unnamed: 0 удалили

In [8]:
data = df.copy()

In [9]:
%%time
data['text'] = data['text'].values.astype('U')

CPU times: user 1.33 s, sys: 2.01 s, total: 3.35 s
Wall time: 4.13 s


In [10]:
# приведём столбец text книжнему регистру
data['text'] = data['text'].str.lower()

In [11]:
# избавимся от ненужных символов, напишем для этого функцию
def clear_text(text):
    subbed = [' '.join(re.sub(r'[^a-zA-Z0-9]', ' ', text[i]).split()) for i in range(len(text))]
    return subbed


In [12]:
%%time
data['text'] = clear_text(data['text'])

CPU times: user 3.33 s, sys: 20.3 ms, total: 3.35 s
Wall time: 3.35 s


In [13]:
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


Wordnet Lemmatizer с соответствующим POS-тегом

In [14]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gazizovilsur26/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
L = WordNetLemmatizer()

In [17]:
# функция выполняет токенизациию и лемматизацию массива текстов c учетом pos_tag и удаление стоп-слов
def get_word_text(corpus):
    corpus_new = []
    for sentence in corpus:
        corpus_new.append(' '.join([L.lemmatize(w, get_wordnet_pos(w))
                                    for w in nltk.word_tokenize(sentence)
                                    if not w in stopwords.words('english')]))
    return corpus_new

In [18]:
%%time
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
 
data['text'] = get_word_text(data['text'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gazizovilsur26/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gazizovilsur26/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gazizovilsur26/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


CPU times: user 19min 27s, sys: 2min 26s, total: 21min 54s
Wall time: 21min 55s


In [19]:
# разделим датасет на тренировочную и тестовую
# данных много, поэтому на обучающую выборку оставим больше данных
train, test = train_test_split(data, test_size=0.25, random_state=12345)

In [20]:
# поделим несэмплированные данные на обучающие признаки и целевые признаки
features_train = train.drop(['toxic'], axis=1)
target_train = train['toxic']
features_test = test.drop(['toxic'], axis=1)
target_test = test['toxic']

*Промежуточный вывод:*
- Привели текст к нижнему регистру
- Избавились от ненужных символов с помощью re.sub
- Сделали лемматизацию с помощью Wordnet Lemmatizer с соответствующим POS-тегом для учёта частей речи
- Разделили датасет на обучающую и тестовую

## Обучение

**CATBOOST**

In [21]:
# обучим модель CatBoost для текстов
model_catboost = CatBoostClassifier(

        #task_type='GPU',
)
param_grid = {
    'iterations': [500, 1000, 1500],
    'learning_rate': [0.05, 0.1, 0.15]
}

catboost = RandomizedSearchCV(
    estimator=model_catboost,
    param_distributions=param_grid,
    scoring='f1',
    cv=3,
    verbose=100)
catboost.fit(features_train,
        target_train,
        text_features=['text'])

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START iterations=500, learning_rate=0.05..........................
0:	learn: 0.6222439	total: 146ms	remaining: 1m 12s
1:	learn: 0.5669670	total: 220ms	remaining: 54.8s
2:	learn: 0.5146987	total: 295ms	remaining: 48.8s
3:	learn: 0.4680552	total: 376ms	remaining: 46.7s
4:	learn: 0.4312677	total: 461ms	remaining: 45.6s
5:	learn: 0.3983684	total: 538ms	remaining: 44.3s
6:	learn: 0.3661322	total: 616ms	remaining: 43.4s
7:	learn: 0.3395001	total: 693ms	remaining: 42.6s
8:	learn: 0.3144725	total: 772ms	remaining: 42.1s
9:	learn: 0.2955340	total: 851ms	remaining: 41.7s
10:	learn: 0.2779834	total: 930ms	remaining: 41.3s
11:	learn: 0.2645534	total: 1.01s	remaining: 41.3s
12:	learn: 0.2541761	total: 1.1s	remaining: 41.3s
13:	learn: 0.2440065	total: 1.18s	remaining: 40.9s
14:	learn: 0.2344013	total: 1.25s	remaining: 40.5s
15:	learn: 0.2262759	total: 1.33s	remaining: 40.3s
16:	learn: 0.2178330	total: 1.41s	remaining: 40s
17:	

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7fb0ece64df0>,
                   param_distributions={'iterations': [500, 1000, 1500],
                                        'learning_rate': [0.05, 0.1, 0.15]},
                   scoring='f1', verbose=100)

In [22]:
print('Лучшие гиперпарметры модели CatBoost:', catboost.best_params_)

Лучшие гиперпарметры модели CatBoost: {'learning_rate': 0.15, 'iterations': 1000}


In [23]:
print('Метрика F1 модели CatBoost на лучших гиперпараметрах:', catboost.best_score_)

Метрика F1 модели CatBoost на лучших гиперпараметрах: 0.7654902009245969


## Тестирование модели

In [24]:
predictions = catboost.predict(features_test)

In [25]:
print('Метрика F1 обученной модели с лучшими гиперпараметрами:', f1_score(target_test, predictions))

Метрика F1 обученной модели с лучшими гиперпараметрами: 0.7803412969283275


## Выводы

**Вывод**
- Подготовка данных:
    - Привели текст к нижнему регистру
    - Избавились от ненужных символов с помощью re.sub
    - Сделали лемматизацию с помощью Wordnet Lemmatizer с соответствующим POS-тегом для учёта частей речи
    - Разделили датасет на обучающую и тестовую
- Обучение:
    - Обучили моель catboost и подобрали лучшие гиперпараметры {'learning_rate': 0.15, 'iterations': 1000}
    - На RandomizedSearch модель показала результат 0.77
- Тестирование модели:
    - На тесте модель catboost с гиперпараметрами {'learning_rate': 0.15, 'iterations': 1000} показала результат 0.78, что удовлетворяет требованию заказчика
- Примечания:
    - Отдельно обученная BERT на валидации показал 0.81 и accuracy 96.6 (хотя accuracy мало о чём говорит при таком дисбалансе классов). 
    - Отдельно обученная Логистическая регрессия на валидации показал 0.72

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны